In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import numpy as np
import scipy
import time
from PIL import Image,ImageOps
from keras.datasets import mnist

from keras import layers
import keras
from keras.layers import Dropout
from matplotlib import pyplot as plt
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import initializers
from keras import optimizers
from matplotlib import pyplot
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.layers.normalization import BatchNormalization
from array import array
import cv2
import os
import random
from google.colab.patches import cv2_imshow



In [ ]:
# if input image is in range 0..1, please first multiply img by 255
# assume image is ndarray of shape [height, width, channels] where channels can be 1, 3 or 4
# get the train, val and test data   
(train_imagesall, train_labels), (test_imagesall, test_labels) = mnist.load_data()

#adding the new dimension for conversion of grayscale to rgb
train_imagesall=train_imagesall[:,:,:,np.newaxis]
test_imagesall=test_imagesall[:,:,:,np.newaxis]
train_images=([])
test_images1=([])

#stacking of the layer vertically about axis =2
for i in range(0,60000):
  #print(a.shape)
  train_images.append(np.concatenate((train_imagesall[i],train_imagesall[i],train_imagesall[i]),axis=2))
for i in range(0,10000):
  test_images1.append(np.concatenate((test_imagesall[i],test_imagesall[i],test_imagesall[i]),axis=2))

train_images=np.array(train_images)
test_images1=np.array(test_images1)

# train_images=train_imagesall.reshape(60000,28,28,1)
# test_images1=test_imagesall.reshape(10000,28,28,1)

val_images = test_images1[0:2000,:,:]
val_labels = test_labels[0:2000]
test_images = test_images1[2000:,:,:]
test_labels = test_labels[2000:]


# display sample
im = train_images[10,:,:]
#plt.imshow(im)
#plt.show()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
# get the dimensions
print("Shape of training images:",train_images.shape)
print("Shape of training labels:",train_labels.shape)
print("unique labels:",np.unique(train_labels))
print("Shape of val images:",val_images.shape)
print("Shape of val labels:",test_labels.shape)

Shape of training images: (60000, 28, 28, 3)
Shape of training labels: (60000,)
unique labels: [0 1 2 3 4 5 6 7 8 9]
Shape of val images: (2000, 28, 28, 3)
Shape of val labels: (8000,)


This is the random network

In [ ]:
#normalising the mean and shifting by the mean
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5
val_images = (val_images / 255) - 0.5
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

# model = Sequential()

# model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(28,28,3)))
# model.add(layers.AveragePooling2D())

# model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
# model.add(layers.AveragePooling2D())

# model.add(layers.Flatten())

# model.add(layers.Dense(units=120, activation='relu'))

# model.add(layers.Dense(units=84, activation='relu'))

# model.add(layers.Dense(units=10, activation = 'softmax'))

#lenet model
model = keras.Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(28,28,3)))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))

model.add(layers.Dense(units=84, activation='relu'))

model.add(layers.Dense(units=10, activation = 'softmax'))

#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
#sgd = optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)


model.compile(
  optimizer=adam,
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# obtain the summary
#saved_model.summary()


es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)
mc = ModelCheckpoint('/content/gdrive/MyDrive/models/task2_mnist_3.h5', monitor='val_accuracy', mode='max', 
                     verbose=1, save_best_only=True)
print(train_images.shape)
print(val_images.shape)
# Train the model.

start_time = time.time()
history=model.fit(
  train_images,
  to_categorical(train_labels),
  validation_data=(val_images, to_categorical(val_labels)),  
  epochs=5,
  batch_size=32,
  shuffle = True,
  callbacks=[es,mc],
)


scores = model.evaluate(
  test_images,
  to_categorical(test_labels)
)

print('Test accuracy:', scores[1])
print("--- %s seconds ---" % (time.time() - start_time))

(60000, 28, 28, 3)
(2000, 28, 28, 3)
Epoch 1/5
1875/1875 [==============================] - 26s 14ms/step - loss: 0.4573 - accuracy: 0.8605 - val_loss: 0.0811 - val_accuracy: 0.9740

Epoch 00001: val_accuracy improved from -inf to 0.97400, saving model to /content/gdrive/MyDrive/models/task2_mnist_3.h5
Epoch 2/5
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0658 - accuracy: 0.9792 - val_loss: 0.0610 - val_accuracy: 0.9800

Epoch 00002: val_accuracy improved from 0.97400 to 0.98000, saving model to /content/gdrive/MyDrive/models/task2_mnist_3.h5
Epoch 3/5
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0458 - accuracy: 0.9858 - val_loss: 0.0752 - val_accuracy: 0.9765

Epoch 00003: val_accuracy did not improve from 0.98000
Epoch 4/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0334 - accuracy: 0.9895 - val_loss: 0.0437 - val_accuracy: 0.9850

Epoch 00004: val_accuracy improved from 0.98000 to 0.98500, saving model to /co

This is the part of pretrained network

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)
mc = ModelCheckpoint('/content/gdrive/MyDrive/models/pretrained_3_mnist.h5', monitor='val_accuracy', mode='max', 
                     verbose=1, save_best_only=True)
saved_model=load_model("/content/gdrive/MyDrive/models/pretrained_3.h5")
start_time = time.time()

history=saved_model.fit(
  train_images,
  to_categorical(train_labels),
  validation_data=(val_images, to_categorical(val_labels)),  
  epochs=5,
  batch_size=32,
  shuffle = True,
  callbacks=[es,mc],
)


scores = saved_model.evaluate(
  test_images,
  to_categorical(test_labels)
)

print('Test accuracy:', scores[1])
print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/5
1875/1875 [==============================] - 25s 13ms/step - loss: 0.2771 - accuracy: 0.9387 - val_loss: 0.1214 - val_accuracy: 0.9620

Epoch 00001: val_accuracy improved from -inf to 0.96200, saving model to /content/gdrive/MyDrive/models/pretrained_3_mnist.h5
Epoch 2/5
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0830 - accuracy: 0.9744 - val_loss: 0.0825 - val_accuracy: 0.9760

Epoch 00002: val_accuracy improved from 0.96200 to 0.97600, saving model to /content/gdrive/MyDrive/models/pretrained_3_mnist.h5
Epoch 3/5
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0626 - accuracy: 0.9817 - val_loss: 0.0735 - val_accuracy: 0.9775

Epoch 00003: val_accuracy improved from 0.97600 to 0.97750, saving model to /content/gdrive/MyDrive/models/pretrained_3_mnist.h5
Epoch 4/5
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0503 - accuracy: 0.9844 - val_loss: 0.0525 - val_accuracy: 0.9840

Epoch 00004: val_accuracy improv